In [1]:
import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

2025-05-05 18:11:09.648 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
st.set_page_config(page_title="Tokyo Stock Market Predictor", layout="wide")
st.title("📈 Tokyo Stock Market Predictor")
st.markdown("Predict future stock prices using machine learning (Random Forest Regressor)")

2025-05-05 18:11:18.124 
  command:

    streamlit run C:\Users\tusha\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [3]:
ticker = st.sidebar.text_input("Enter Tokyo Stock Ticker (e.g. 7203.T for Toyota):", value="7203.T")
start_date = st.sidebar.date_input("Start Date", value=pd.to_datetime("2015-01-01"))
end_date = st.sidebar.date_input("End Date", value=pd.to_datetime("2024-12-31"))


In [6]:
def load_data(ticker, start, end):
    df = yf.download(ticker, start=start, end=end)
    df = df[["Open", "High", "Low", "Close", "Volume"]]
    df["SMA_10"] = df["Close"].rolling(window=10).mean()
    df["SMA_50"] = df["Close"].rolling(window=50).mean()
    df["Return"] = df["Close"].pct_change()
    df = df.dropna()
    return df

df = load_data(ticker, start_date, end_date)


[*********************100%***********************]  1 of 1 completed


In [7]:
st.subheader(" Historical Stock Data")
st.dataframe(df.tail())


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [8]:
features = ["Open", "High", "Low", "Volume", "SMA_10", "SMA_50", "Return"]
target = "Close"

X = df[features]
y = df[target]

In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)


In [11]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

C:\Users\tusha\AppData\Local\Temp/ipykernel_36840/650042473.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [12]:
predictions = model.predict(X_test)

In [13]:
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)


In [16]:
st.subheader(" Model Performance")
st.write(f"**Mean Squared Error:** {mse:.2f}")
st.write(f"**R² Score:** {r2:.2f}")

In [21]:

plot_df = pd.DataFrame({"Actual": y_test.values.ravel(),"Predicted": predictions.ravel()}).reset_index(drop=True)
st.line_chart(plot_df)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [22]:
csv = plot_df.to_csv(index=False).encode('utf-8')
st.download_button(label="Download Prediction CSV",data=csv,file_name="tokyo_stock_predictions.csv",mime="text/csv")

False

In [23]:
st.markdown("---")
st.markdown("Made with using [Streamlit](https://streamlit.io/) and [Yahoo Finance](https://finance.yahoo.com/)")

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)